<img src="http://www.cs.wm.edu/~rml/images/wm_horizontal_single_line_full_color.png">

<h1 style="text-align:center;">CSCI 140</h1>
<h1 style="text-align:center;">
Using Series and Data Frames
</h1>

Previous: [Data_Frames](Data_Frames)

Let's look at Series and Data Frames in action with a **real** (and real small) data set! First we need some import lines:

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

`%matplotlib inline` is necessary in the notebook to draw plots - you do not need this if you are working from the command line. We are importing `pandas` so that we can use Series and Data Frames. `numpy` provides numerical operations and accesses to NaN. The `seaborn` and `matplotlib.pyplot` modules provide graphic capability.

We are going to work with a very small data set so that we can look at all of the data in the notebook. In practice, data sets will be large enough that you cannot possibly look at all of the data manually.

In [3]:
nd = pd.read_csv('NDHUB.EarthquakeLocations.csv')

We can use the `head()` method to see the first few rows of the data frame:

In [4]:
nd.head()

,OBJECTID\tLONGITUDE\tLATITUDE\tEVENTDATE\tMAGNITUDE\tINTENSITY\tDEPTH\tAREA\tYEAR_OCCURRED
0,641\t-103.6\t48.2\t19150808\t3.7\tIV\tU\tWilli...
1,642\t-102.1\t46.9\t19270430\t3.2\tIII\tU\tHebr...
2,643\t-103.7\t48.2\t19461026\t3.7\tIV\tU\tWilli...
3,644\t-100.9\t46\t19470514\t3.7\tIV\tU\tSelfrid...
4,645\t-100.742\t46.588\t19680708\t4.4\tIV\t20.5...


Hmmm, this file had a .csv extention but when we read it in, it looks like it is tab separated. Notice that most of the data is mushed into one column, and we can see `\t` which indicates a tab. Let's try to read it in adding an optional argument:

In [5]:
nd = pd.read_csv('NDHUB.EarthquakeLocations.csv', sep = '\t')

In [6]:
nd.head()

,OBJECTID,LONGITUDE,LATITUDE,EVENTDATE,MAGNITUDE,INTENSITY,DEPTH,AREA,YEAR_OCCURRED
0,641,-103.600,48.200,19150808,3.7,IV,U,Williston,1915
1,642,-102.100,46.900,19270430,3.2,III,U,Hebron,1927
2,643,-103.700,48.200,19461026,3.7,IV,U,Williston,1946
3,644,-100.900,46.000,19470514,3.7,IV,U,Selfridge,1947
4,645,-100.742,46.588,19680708,4.4,IV,20.5 (33),Huff,1968


That's better! The `sep` argument tells the function how our data is delimited. A delimiter is the character that separates columns of data, in this example the delimiter is a tab.

We know that each row and column in the data frame is an object of type Series:

In [7]:
print(type(nd['OBJECTID']))
print(type(nd.iloc[0]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


The individual columns also have a data type depending on what type of information they store. We can see these by looking at the `dtype` attribute for a Series or `dtypes` for a data frame:

In [8]:
print(nd.dtypes)

OBJECTID           int64
LONGITUDE        float64
LATITUDE         float64
EVENTDATE          int64
MAGNITUDE        float64
INTENSITY         object
DEPTH             object
AREA              object
YEAR_OCCURRED      int64
dtype: object


In [9]:
print(nd['YEAR_OCCURRED'].dtype)

int64


Hmm, so what do we think of these data types? `int64` indicates an integer, `float64` indicates a float, and `object` can be various things - most often this will refer to a string or a column with mixed data types (which is generally something we want to fix). 

`EVENTDATE` isn't really an integer - it's just a date expressed in a wonky format. We should convert this to a `datetime` or at least a string. Let's turn these values into a column of strings:

In [10]:
nd['EVENTDATE'] = nd['EVENTDATE'].astype('str')
print(nd.dtypes)

OBJECTID           int64
LONGITUDE        float64
LATITUDE         float64
EVENTDATE         object
MAGNITUDE        float64
INTENSITY         object
DEPTH             object
AREA              object
YEAR_OCCURRED      int64
dtype: object


It is tempting to try to do something like:

`str(nd['EVENTDATE']`

That is how we've cast variables before. However, a Series is a collection of objects - casting a collection to a string like this does not work. It doesn't call an error, but it calls the `__str__` method for that object. We get a **string representation of the object** but it doesn't convert the individual items in the series to strings. The `as_type` method allows us to cast every item in a Series to a particular type.

Series objects also support vectorized string operations. Huh? 

Suppose that you want to turn all of the values in the AREA column into uppercase strings. You might try something like this:

In [11]:
nd['AREA'] = nd['AREA'].upper()

AttributeError: 'Series' object has no attribute 'upper'

This doesn't work because `nd['AREA']` is a Series, NOT a string. It doesn't have an `upper` method. We really want to look at every item in the Series and convert them one by one to uppercase. You **could** do this with a for loop, but it's easier to do it this way:

In [12]:
nd['AREA'] = nd['AREA'].str.upper()
nd.head()

,OBJECTID,LONGITUDE,LATITUDE,EVENTDATE,MAGNITUDE,INTENSITY,DEPTH,AREA,YEAR_OCCURRED
0,641,-103.600,48.200,19150808,3.7,IV,U,WILLISTON,1915
1,642,-102.100,46.900,19270430,3.2,III,U,HEBRON,1927
2,643,-103.700,48.200,19461026,3.7,IV,U,WILLISTON,1946
3,644,-100.900,46.000,19470514,3.7,IV,U,SELFRIDGE,1947
4,645,-100.742,46.588,19680708,4.4,IV,20.5 (33),HUFF,1968


Adding the `str` after the Series allows us to use the vectorized operation that will work on each item individually. There are many other string methods that you can invoke on a Series this way:

In [13]:
help(pd.Series.str)

Help on class StringMethods in module pandas.core.strings:

class StringMethods(pandas.core.base.NoNewAttributesMixin)
 |  StringMethods(data)
 |  
 |  Vectorized string functions for Series and Index. NAs stay NA unless
 |  handled otherwise by a particular method. Patterned after Python's string
 |  methods, with some inspiration from R's stringr package.
 |  
 |  Examples
 |  --------
 |  >>> s.str.split('_')
 |  >>> s.str.replace('_', '')
 |  
 |  Method resolution order:
 |      StringMethods
 |      pandas.core.base.NoNewAttributesMixin
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, key)
 |  
 |  __init__(self, data)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |  
 |  capitalize(self)
 |      Convert strings in the Series/Index to be capitalized.
 |      
 |      Equivalent to :meth:`str.capitalize`.
 |      
 |      Returns
 |      -------
 |      Series or Index of object
 |      
 |      See

What if you want to carry out an operation on a Series that is not part of the Series `StringMethods`? Or what if you want to carry out an operation on a Series of floats or ints? You can use `apply` to accomplish this. Let's re-format `EVENT_DATE`. How would we do this for a single string?

In [14]:
def reformat(date): #expect date to be YYYYMMDD
    year = date[:4]
    month = date[4:6]
    day = date[6:8]
    return day + '.' + month + '.' + year

OK, we've got a function that works on one string. We will use `apply` to apply it to a whole Series:

In [15]:
result = nd['EVENTDATE'].apply(reformat)
print(result)
nd['EVENTDATE'] = result

0     08.08.1915
1     30.04.1927
2     26.10.1946
3     14.05.1947
4     08.07.1968
5     09.03.1982
6     11.11.1998
7     15.11.2008
8     03.01.2009
9     28.09.2012
10    14.06.2010
11    31.08.2009
12    21.03.2010
Name: EVENTDATE, dtype: object


What `apply` does is to submit each item from the Series sequentially as an argument to the function. The **function name** is the argument we send to `apply`. Because `apply` is a Series method, it can access the internal data of the Series, which is the individual items.

You can apply any valid function, including built-in and imported ones:

In [16]:
edited_lat = nd['LATITUDE'].apply(round)
print(edited_lat)

0     48
1     47
2     48
3     46
4     47
5     49
6     49
7     47
8     48
9     48
10    46
11    48
12    48
Name: LATITUDE, dtype: int64


This isn't something we actually want in our data frame, so we won't overwrite the LATITUDE column with this new Series.

What are some things we might like to do to facilitate analysis?

1) Convert INTENSITY to a Series of integers

2) Process DEPTH: currently the depth is provided as miles and kilometers, and some values are just marked as 'U'! Let's just convert them all to miles, and change anything that is 'U' to missing.

Let's think about the steps to achieve aim 1:

-Map each Roman Numeral to the appropriate number
-Change each item in the series to an integer

There are a few ways to do this. The easiest is probably to use the `replace` method, which will accept a dictionary as an argument:

In [17]:
roman = {'I':1, 'II':2, 'III':3, 'IV':4}
result = nd['INTENSITY'].replace(roman)
print(result)
nd['INTENSITY'] = result

0     4
1     3
2     4
3     4
4     4
5     3
6     4
7     2
8     1
9     3
10    1
11    1
12    2
Name: INTENSITY, dtype: int64


In [18]:
print(nd['INTENSITY'].dtype)
nd.head()

int64


,OBJECTID,LONGITUDE,LATITUDE,EVENTDATE,MAGNITUDE,INTENSITY,DEPTH,AREA,YEAR_OCCURRED
0,641,-103.600,48.200,08.08.1915,3.7,4,U,WILLISTON,1915
1,642,-102.100,46.900,30.04.1927,3.2,3,U,HEBRON,1927
2,643,-103.700,48.200,26.10.1946,3.7,4,U,WILLISTON,1946
3,644,-100.900,46.000,14.05.1947,3.7,4,U,SELFRIDGE,1947
4,645,-100.742,46.588,08.07.1968,4.4,4,20.5 (33),HUFF,1968


How do we deal with the depth? 

The `U` values are actually missing data. There is not enough information to determine why they are missing (it couldn't be measured? it wasn't recorded? Notice that these are pre-1960). We don't necessarily want to delete the entire observation, but we would like to mark these as missing.

The way we typically do this when working with `pandas` is to set missing values to the value `numpy.nan`. This value does not cause an error, but rather is ignored when mathematical calculations are performed on a column. The value `numpy.nan` is considered a `float` so it can cause problems in a column of strings....

Notice that the rest of the column is actually filled with strings since there are two measurements - the first is in miles, the second is in kilometers. It would be nice to separate these out and maybe make two columns.

Let's think about how we would do this for a single string like **8.3 (13.4)**. The steps might be:

-Split on the space using `split`

-Save the first item in the list as a float called miles unless the data is missing

-Replace missing data ('U') with np.nan

For example, when the data is not missing:

In [19]:
data = '8.3 (13.4)'.split()
miles = float(data[0])
print(miles)

8.3


Consider that for data coded as 'U', when we split into a list, the list will only have one item in it. Let's adapt this to work with a Series:

In [24]:
def get_miles(x):
    x = x.split()
    if len(x) >1:
        return float(x[0])
    else:
        return np.nan

Before we apply this to the Series, let's test it out:

In [25]:
print(get_miles('8.3 (12.4)'))
print(get_miles('U'))

8.3
nan


Now we will apply it to the Series:

In [26]:
miles = nd['DEPTH'].apply(get_miles)
miles

0      NaN
1      NaN
2      NaN
3      NaN
4     20.5
5     11.2
6      3.1
7     11.2
8      8.3
9      0.4
10     3.1
11     3.1
12     3.1
Name: DEPTH, dtype: float64

This creates the new Series, but how would we add it to the Data Frame? There's a few ways. We could just do this:

nd['DEPTH_MILES'] = miles

This creates a new column called DEPTH_MILES and puts it at the END - that is, the last column. What if we don't want it to be last, maybe we want it to be the second column.

We could use the insert method:

In [ ]:
nd.insert(column = 'DEPTH_MILES', value = miles, loc=1)

In [ ]:
nd

One more thing...`np.nan` is quite peculiar:

In [ ]:
var = np.nan
print(var == np.nan)

Huh? What? That's right `np.nan` does not equal `np.nan` - these represent unknown quantities which may or may not be equivalent. You can test if a value is `np.nan` like this:

In [ ]:
var = np.nan
print(np.isnan(var))
var_2 = 5
print(np.isnan(var_2))
var_3 = [5,3,1,2]
print(np.isnan(var_3))
var = np.nan
print(pd.isna(var))
print(pd.isna(var_2))
print(pd.isna(var_3))

Notice that `np.isnan` and `pd.isnull` will work on a list(and other list-like objects), returning a value for each item in the list. 

Rembmer that `np.nan` is a numerical quantity. The function `np.isnan` gives you an error if you try to test strings. However, `pd.isna` will work:

In [ ]:
var = 'a string'
print(pd.isna(var))
print(np.isnan(var))

One last thing. Perhaps we'd like to make one of our columns the index.

Let's see what the unique values are for `YEAR_OCCURED`:

In [ ]:
nd['YEAR_OCCURRED'].unique()

We can see if we have any duplicates using `value_counts`. This returns a Series where the index is each unique value and the data in the Series is how many times that value occured:

In [ ]:
nd['YEAR_OCCURRED'].value_counts()

Duplicate indices are permitted in both Data Frames and Series, but it can be confusing. Let's use the EVENTDATE column as the index instead:

In [ ]:
nd = nd.set_index('EVENTDATE')
nd.head()